### 1. Configuração Inicial

**Objetivo:** Definir todas as variáveis de caminhos, URLs e parâmetros que usaremos. Manter tudo em uma única célula facilita a manutenção.

In [0]:
# --- CONFIGURAÇÃO DA FONTE DE DADOS ---
data_source = "sinasc"
csv_url = "https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SINASC/csv/SINASC_2024.csv"

# --- CONTROLE DE LIMITAÇÃO DE CARGA ---
sample_rows = 10000
sample_file_name = f"SINASC_2024_SAMPLE_{sample_rows}.csv"

# --- CONFIGURAÇÃO DOS CAMINHOS ---
root_path = "/lakehouse/health_insights_brasil"
# Usaremos apenas um diretório na landing zone, que conterá nossa amostra
landing_zone_path = f"{root_path}/landing_zone/{data_source}"

print("✅ Configuração concluída.")
print(f"   - O arquivo de amostra será salvo em: {landing_zone_path}")

✅ Configuração concluída.
   - O arquivo de amostra será salvo em: /lakehouse/health_insights_brasil/landing_zone/sinasc


### 2. Limpeza do Ambiente

**Objetivo:** Remover os dados de execuções anteriores para garantir uma carga limpa e idempotente.

In [0]:
print(f"Limpando o diretório de destino: {landing_zone_path}")
dbutils.fs.rm(landing_zone_path, recurse=True)
dbutils.fs.mkdirs(landing_zone_path)
print("✅ Diretório limpo e pronto para a ingestão.")

Limpando o diretório de destino: /lakehouse/health_insights_brasil/landing_zone/sinasc
✅ Diretório limpo e pronto para a ingestão.


### 3. Download Parcial do Arquivo

**Objetivo:** Usar o comando `curl` para baixar apenas os primeiros 20MB do arquivo. Isso é muito mais rápido que baixar o arquivo inteiro e é mais que suficiente para extrairmos 10.000 linhas.

-   `curl`: Ferramenta de linha de comando para transferir dados.
-   `-r 0-20000000`: Especifica o intervalo de bytes para baixar (do byte 0 ao 20.000.000).
-   `-o`: Especifica o nome do arquivo de saída.

In [0]:
%sh

# Baixamos apenas um "pedaço" do arquivo para um arquivo temporário
curl -r 0-20000000 -o "/tmp/SINASC_PARTIAL.csv" "https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SINASC/csv/SINASC_2024.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0 19.0M    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  2 19.0M    2  474k    0     0   310k      0  0:01:02  0:00:01  0:01:01  310k
 64 19.0M   64 12.2M    0     0  5126k      0  0:00:03  0:00:02  0:00:01 5124k
100 19.0M  100 19.0M    0     0  6716k      0  0:00:02  0:00:02 --:--:-- 6714k


### 4. Extrair Amostra Limpa do Arquivo Parcial

**Objetivo:** Agora que temos um pedaço do arquivo, usamos o comando `head` para pegar o número exato de linhas que queremos (cabeçalho + 10.000 linhas) e salvar em nosso arquivo de amostra final. Isso garante que não teremos uma linha cortada no final.

In [0]:
%sh

# Usamos o head para garantir que temos o número exato de linhas
# O resultado é o nosso arquivo de amostra final e limpo
head -n 10001 "/tmp/SINASC_PARTIAL.csv" > "/tmp/SINASC_2024_SAMPLE_10000.csv"

### 5. Mover Arquivo de Amostra Final para a Landing Zone

**Objetivo:** Mover nosso arquivo de amostra, limpo e com o tamanho certo, para o DBFS para que possa ser usado pelo próximo notebook.

In [0]:
source_sample_path = f"file:/tmp/{sample_file_name}"
destination_sample_path = f"{landing_zone_path}/{sample_file_name}"

print(f"Movendo arquivo de amostra para: {destination_sample_path}")
dbutils.fs.mv(source_sample_path, destination_sample_path)

print("✅ Arquivo de amostra salvo na Landing Zone com sucesso.")

Movendo arquivo de amostra para: /lakehouse/health_insights_brasil/landing_zone/sinasc/SINASC_2024_SAMPLE_10000.csv
✅ Arquivo de amostra salvo na Landing Zone com sucesso.


### 6. Verificação Final

**Objetivo:** Listar o arquivo na Landing Zone e ver as primeiras linhas para confirmar que tudo funcionou.

In [0]:
print("--- Conteúdo do diretório da LANDING ZONE (AMOSTRA) ---")
display(dbutils.fs.ls(landing_zone_path))

print("\n--- Amostra do Conteúdo do Arquivo Final ---")
# O comando 'head' do dbutils lê o início de um arquivo no DBFS
print(dbutils.fs.head(destination_sample_path))

--- Conteúdo do diretório da LANDING ZONE (AMOSTRA) ---


path,name,size,modificationTime
dbfs:/lakehouse/health_insights_brasil/landing_zone/sinasc/SINASC_2024_SAMPLE_10000.csv,SINASC_2024_SAMPLE_10000.csv,3453302,1755206859000



--- Amostra do Conteúdo do Arquivo Final ---
[Truncated to first 65536 bytes]
"contador";"ORIGEM";"CODESTAB";"CODMUNNASC";"LOCNASC";"IDADEMAE";"ESTCIVMAE";"ESCMAE";"CODOCUPMAE";"QTDFILVIVO";"QTDFILMORT";"CODMUNRES";"GESTACAO";"GRAVIDEZ";"PARTO";"CONSULTAS";"DTNASC";"HORANASC";"SEXO";"APGAR1";"APGAR5";"RACACOR";"PESO";"IDANOMAL";"DTCADASTRO";"CODANOMAL";"NUMEROLOTE";"VERSAOSIST";"DTRECEBIM";"DIFDATA";"OPORT_DN";"DTRECORIGA";"NATURALMAE";"CODMUNNATU";"CODUFNATU";"ESCMAE2010";"SERIESCMAE";"DTNASCMAE";"RACACORMAE";"QTDGESTANT";"QTDPARTNOR";"QTDPARTCES";"IDADEPAI";"DTULTMENST";"SEMAGESTAC";"TPMETESTIM";"CONSPRENAT";"MESPRENAT";"TPAPRESENT";"STTRABPART";"STCESPARTO";"TPNASCASSI";"TPFUNCRESP";"TPDOCRESP";"DTDECLARAC";"ESCMAEAGR1";"STDNEPIDEM";"STDNNOVA";"CODPAISRES";"TPROBSON";"PARIDADE";"KOTELCHUCK"
1;"1";"2516500";"110001";"1";"24";"1";"3";"999992";"01";"00";"110001";"5";"1";"2";"2";"14022024";"0845";"1";"08";"09";"4";"3120";"2";"05032024";;"20240003";"3.2.50";"05032024";"020";"20";"050320